In [4]:
import pandas as pd
import numpy as np
from collections import *
from itertools import islice
from datetime import datetime
import pickle
import os
import time
import csv
import logging
import threading
from multiprocessing import Process, Pool, Manager
import multiprocessing as mp
import os

In [5]:
with open('./CTU/CTU_42/conn.log') as f:
    f = f.readlines()
for line in f:
    if line.split("\t")[0]=='#fields':
        columns = line.split("\t")[1:]
        for i, c in enumerate(columns):
            if len(c.split("\n"))>1:
                columns[i] = c.split("\n")[0]
        break

In [8]:
df1 = pd.read_csv('./CTU/CTU_42/' +'conn.log', delimiter="\t", header=None, names=columns, skiprows=lambda x:x in range(8), skipfooter=1)
df2 = pd.read_csv('./CTU/CTU_43/' +'conn.log', delimiter="\t", header=None, names=columns, skiprows=lambda x:x in range(8), skipfooter=1)
df3 = pd.read_csv('./CTU/CTU_50/' +'conn.log', delimiter="\t", header=None, names=columns, skiprows=lambda x:x in range(8), skipfooter=1)

/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  
/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
len(df1)

4070899

In [10]:
len(df2)

2565282

In [12]:
total_df = pd.concat([df1,df2,df3])

In [17]:
infected = ['147.32.84.165', '147.32.84.191', '147.32.84.192', '147.32.84.193', '147.32.84.204', '147.32.84.205',\
            '147.32.84.206', '147.32.84.207', '147.32.84.208', '147.32.84.209']

In [19]:
total_df["class"] = 0

In [116]:
len(total_df[~total_df["id.orig_h"].isin(infected)])

9053459

In [117]:
len(total_df[total_df["id.orig_h"].isin(infected)])

208514

In [26]:
total_df.loc[total_df["id.orig_h"].isin(infected),["class"]] = "botnet"

In [28]:
total_df.loc[~total_df["id.orig_h"].isin(infected),["class"]] = "normal"

In [30]:
total_df.reset_index(inplace=True)

In [31]:
total_df[total_df["id.orig_h"].isin(infected)]

,index,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,class
0,0,1.312962e+09,Cm4Iyt3EZ8wcV3lhvk,68.52.88.162,49883,147.32.84.59,6881,tcp,-,0.000016,...,-,-,0,Sr,1,52,1,40,(empty),normal
1,1,1.312962e+09,CKkgnx1IoiPMY51uF2,82.7.84.251,49918,147.32.84.59,6881,tcp,-,0.000016,...,-,-,0,Sr,1,52,1,40,(empty),normal
2,2,1.312962e+09,COaxDR2ImPuv2lXxui,68.52.88.162,49883,147.32.84.59,6881,tcp,-,0.000016,...,-,-,0,Sr,1,52,1,40,(empty),normal
3,3,1.312962e+09,CDn3aA3EPDliE9ZPJb,82.7.84.251,49918,147.32.84.59,6881,tcp,-,0.000121,...,-,-,0,Sr,1,52,1,40,(empty),normal
4,4,1.312962e+09,C5h5vWjIYesiHr6,147.32.86.194,2063,217.163.21.35,80,tcp,-,0.000326,...,-,-,0,fA,1,40,1,40,(empty),normal
5,5,1.312962e+09,CYJRZ54aNSEli4Y55c,83.3.77.74,32882,147.32.85.5,21857,tcp,-,0.000154,...,-,-,0,FfA,2,80,1,40,(empty),normal
6,6,1.312962e+09,CvT8o91GKgX2u02Qjj,147.32.84.59,49382,74.200.246.228,80,tcp,-,0.000307,...,-,-,0,fAR,2,80,1,40,(empty),normal
7,7,1.312962e+09,Cap7B92GV4mqOrUWn5,147.32.86.194,2060,77.238.167.32,80,tcp,-,0.000258,...,-,-,0,fA,1,40,1,40,(empty),normal
8,8,1.312962e+09,C72CSLMgY89augBl8,147.32.86.141,2019,178.236.4.29,80,tcp,-,4.025889,...,-,-,0,f,0,0,2,80,(empty),normal
9,9,1.312962e+09,CIc51c44pAN97ywGWe,147.32.86.141,2017,178.236.4.29,80,tcp,-,4.025845,...,-,-,0,f,0,0,2,80,(empty),normal


In [137]:
Counter(total_df["class"])

Counter({'normal': 9053459, 'botnet': 208514})

In [32]:
total_df.to_csv('ctu_129_bro_result.csv')

In [39]:
ss = pd.to_datetime(total_df[:10]["ts"], unit='s')

In [62]:
total_df.columns

Index(['index', 'ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h',
       'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
       'tunnel_parents', 'class', 'Date'],
      dtype='object')

In [118]:
Counter(total_df["class"])

Counter({'normal': 9053459, 'botnet': 208514})

In [125]:
total_df[:100]["starttime"] = pd.to_datetime(total_df[:100].ts)

/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [127]:
total_df[:100]

,index,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,...,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,class,Date,starttime
0,0,1.312962e+09,Cm4Iyt3EZ8wcV3lhvk,68.52.88.162,49883,147.32.84.59,6881,tcp,-,0.000016,...,0,Sr,1,52,1,40,(empty),normal,2011-08-10 07:46:55.742159843,1970-01-01 00:00:01.312962415
1,1,1.312962e+09,CKkgnx1IoiPMY51uF2,82.7.84.251,49918,147.32.84.59,6881,tcp,-,0.000016,...,0,Sr,1,52,1,40,(empty),normal,2011-08-10 07:46:56.066534042,1970-01-01 00:00:01.312962416
2,2,1.312962e+09,COaxDR2ImPuv2lXxui,68.52.88.162,49883,147.32.84.59,6881,tcp,-,0.000016,...,0,Sr,1,52,1,40,(empty),normal,2011-08-10 07:46:56.413175106,1970-01-01 00:00:01.312962416
3,3,1.312962e+09,CDn3aA3EPDliE9ZPJb,82.7.84.251,49918,147.32.84.59,6881,tcp,-,0.000121,...,0,Sr,1,52,1,40,(empty),normal,2011-08-10 07:46:56.616441011,1970-01-01 00:00:01.312962416
4,4,1.312962e+09,C5h5vWjIYesiHr6,147.32.86.194,2063,217.163.21.35,80,tcp,-,0.000326,...,0,fA,1,40,1,40,(empty),normal,2011-08-10 07:46:53.049894810,1970-01-01 00:00:01.312962413
5,5,1.312962e+09,CYJRZ54aNSEli4Y55c,83.3.77.74,32882,147.32.85.5,21857,tcp,-,0.000154,...,0,FfA,2,80,1,40,(empty),normal,2011-08-10 07:46:53.053771019,1970-01-01 00:00:01.312962413
6,6,1.312962e+09,CvT8o91GKgX2u02Qjj,147.32.84.59,49382,74.200.246.228,80,tcp,-,0.000307,...,0,fAR,2,80,1,40,(empty),normal,2011-08-10 07:46:53.082381010,1970-01-01 00:00:01.312962413
7,7,1.312962e+09,Cap7B92GV4mqOrUWn5,147.32.86.194,2060,77.238.167.32,80,tcp,-,0.000258,...,0,fA,1,40,1,40,(empty),normal,2011-08-10 07:46:53.087248087,1970-01-01 00:00:01.312962413
8,8,1.312962e+09,C72CSLMgY89augBl8,147.32.86.141,2019,178.236.4.29,80,tcp,-,4.025889,...,0,f,0,0,2,80,(empty),normal,2011-08-10 07:46:53.138033867,1970-01-01 00:00:01.312962413
9,9,1.312962e+09,CIc51c44pAN97ywGWe,147.32.86.141,2017,178.236.4.29,80,tcp,-,4.025845,...,0,f,0,0,2,80,(empty),normal,2011-08-10 07:46:53.171248913,1970-01-01 00:00:01.312962413


In [130]:
def preprocess_stat_2_CTU(df, period_len, rm_ntp):
#     df = df.loc[~df["service"].isin(['mysql', 'imap','ftp','ftp-data'])]
    df["orig_bytes"] = df["orig_bytes"].apply(lambda x:0 if x=='-' else x)
    df["resp_bytes"] = df["orig_bytes"].apply(lambda x:0 if x=='-' else x)
    df["orig_bytes"] = df["orig_bytes"].astype('int64')
    df["resp_bytes"] = df["resp_bytes"].astype('int64')
    if rm_ntp:
        df = df.loc[~df["service"].isin(['ntp'])]
    df = df.sort_values(by='ts')
    df = df.reset_index(drop=True)
    df["StartTime"] = pd.to_datetime(df.ts, unit='s')
    df["min"] = df["StartTime"].apply(lambda x: x.minute)
    df["sec"] = df["StartTime"].apply(lambda x: x.second)
    df["hour"] = df["StartTime"].apply(lambda x: x.hour)
    df["time_bin"] = df.apply(lambda x:int(x["hour"]) * 60 * 60 +int(x["min"]) * 60 + int(x["sec"]), axis=1)
    tb_lst = df.time_bin.unique()
    tb_dict = dict()
    for i in tb_lst:
        tb_dict[i] = (i - df.time_bin.min())//period_len
    df["time_chunk"] = df["time_bin"].apply(lambda x:tb_dict[x])
    # df = port_service_rate(df)
    df_class = df.filter(["time_chunk", "id.orig_h", "class"], axis=1)
    df.duration = df.apply(lambda x: 0 if x.duration =='-' else x.duration, axis=1)
    df.duration = df.duration.astype(float)
    df_filtered = df.filter(['duration','orig_pkts','orig_ip_bytes','orig_bytes', 'resp_bytes',\
                             'missed_bytes','resp_pkts', 'resp_ip_bytes','proto','service',\
                             "time_chunk",'id.orig_h', 'id.resp_p', 'id.orig_p','id.resp_h'], axis=1)

    df_dport = df_filtered.groupby(["time_chunk","id.orig_h"])["id.resp_p"].nunique().reset_index(level=[0,1])
    df_dstaddr = df_filtered.groupby(["time_chunk","id.orig_h"])["id.resp_h"].nunique().reset_index(level=[0,1])
    df_sport = df_filtered.groupby(["time_chunk","id.orig_h"])["id.orig_p"].nunique().reset_index(level=[0,1])
    
    grouped = df_filtered.groupby(["time_chunk","id.orig_h"])
    df_ipcnt = pd.DataFrame(grouped["id.orig_h"].agg('count')).rename(columns={'id.orig_h':"sripcnt"}).reset_index()
    result = grouped['duration','orig_pkts','orig_ip_bytes',\
                     'missed_bytes','resp_pkts','resp_ip_bytes','orig_bytes','resp_bytes'].aggregate([np.mean, np.std])
    result_service = df_filtered.groupby(["time_chunk","id.orig_h","service"]).size().unstack(fill_value=0).reset_index(level=[0,1])
    result_service = result_service.drop(["-"], axis=1)
    result_proto = df_filtered.groupby(["time_chunk","id.orig_h","proto"]).size().unstack(fill_value=0).reset_index(level=[0,1])
    # result_state = df_filtered.groupby(["time_chunk","SrcAddr","State"]).size().unstack(fill_value=0).reset_index(level=[0,1])
    
    df_merged = result.reset_index(level=[0,1])
    df_merged.columns = [' '.join(col).strip() for col in df_merged.columns.values]
    df_merged = df_merged.drop_duplicates()
    df_merged = df_merged.fillna(0)
    df_merged = df_merged.merge(df_class.drop_duplicates(subset=["id.orig_h","time_chunk"]), how='left')
    df_final = result_service.merge(df_merged)
    df_final = df_final.merge(result_proto)
    df_final = df_final.merge(df_dport)
    df_final = df_final.merge(df_dstaddr)
    df_final = df_final.merge(df_sport)
    df_final = df_final.merge(df_ipcnt)
    # df_final = df_final.merge(result_state)
    df_fill_oh = df_final.dropna()
    normalize_column = ["duration mean","duration std", "orig_pkts mean", \
                        "orig_pkts std", "orig_ip_bytes mean", "orig_ip_bytes std",\
                        "orig_bytes mean", "orig_bytes std", "resp_bytes mean", "resp_bytes std",\
                        "missed_bytes mean", "missed_bytes std",\
                        "resp_pkts mean", "resp_pkts std", "resp_ip_bytes mean", "resp_ip_bytes std"]\
                        + ["id.resp_p", "id.resp_h","id.orig_p","sripcnt"]
    normalize_column = [x for x in normalize_column if str(x) != 'nan']
    for i in normalize_column:
        df_fill_oh[i] = (df_fill_oh[i] - df_fill_oh[i].min())/(df_fill_oh[i].max()-df_fill_oh[i].min())

    normalize_column = list(df.service.unique())
    normalize_column.remove("-")
    normalize_column = [x for x in normalize_column if str(x) != 'nan']
    df_fill_oh.loc[:, normalize_column] =\
             df_fill_oh.loc[:,normalize_column].div(df_fill_oh.loc[:,normalize_column].sum(axis=1), axis=0)
    normalize_column = ['icmp', 'tcp', 'udp', 'arp', 'igmp', 'rtp']
    for c in normalize_column:
        if c not in df_fill_oh.columns:
            df_fill_oh[c] = 0
    df_fill_oh.loc[:, normalize_column] =\
             df_fill_oh.loc[:,normalize_column].div(df_fill_oh.loc[:,normalize_column].sum(axis=1), axis=0)
    df_fill_oh = df_fill_oh.fillna(0)
    df_fill_oh = df_fill_oh.sort_index(axis=1)
    return df_fill_oh

In [131]:
a = preprocess_stat_2_CTU(total_df, 60, False)

In [133]:
a["time_chunk"].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [143]:
len(a[a["class"]=='normal']["id.orig_h"].unique())

1141915

In [144]:
len(a[a["class"]=='botnet']["id.orig_h"].unique())

10

In [106]:
len(a.columns)

35

In [52]:
total_df.service.unique()

array(['-', 'http', 'ssl', 'dns', 'pop3', 'snmp', 'smtp', 'http,ssl',
       'sip', 'ftp-data', 'smtp,ssl', 'teredo', 'ftp', 'dhcp', 'ssh',
       'rdp', 'radius', 'http,irc', 'irc', 'socks', 'irc,http',
       'ssl,http', 'ssl,smtp'], dtype=object)

In [54]:
total_df.proto.unique()

array(['tcp', 'udp', 'icmp'], dtype=object)

In [7]:
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents
0,1.312962e+09,Cm4Iyt3EZ8wcV3lhvk,68.52.88.162,49883,147.32.84.59,6881,tcp,-,0.000016,0,...,REJ,-,-,0,Sr,1,52,1,40,(empty)
1,1.312962e+09,CKkgnx1IoiPMY51uF2,82.7.84.251,49918,147.32.84.59,6881,tcp,-,0.000016,0,...,REJ,-,-,0,Sr,1,52,1,40,(empty)
2,1.312962e+09,COaxDR2ImPuv2lXxui,68.52.88.162,49883,147.32.84.59,6881,tcp,-,0.000016,0,...,REJ,-,-,0,Sr,1,52,1,40,(empty)
3,1.312962e+09,CDn3aA3EPDliE9ZPJb,82.7.84.251,49918,147.32.84.59,6881,tcp,-,0.000121,0,...,REJ,-,-,0,Sr,1,52,1,40,(empty)
4,1.312962e+09,C5h5vWjIYesiHr6,147.32.86.194,2063,217.163.21.35,80,tcp,-,0.000326,0,...,SHR,-,-,0,fA,1,40,1,40,(empty)


In [13]:
len(df)

4070899

In [28]:
df["date"] = pd.to_datetime(df["ts"], unit='s')
# df["date"] = df["date"].dt.strftime('%Y-%m-%d:%h-%m-%s')

In [29]:
df = df.sort_values(by=['date'])

In [55]:
df[1373050:1373100]

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,date
1373244,1.312970e+09,CTChAD2G9mnR8PYJR6,69.198.113.18,1583,147.32.84.229,13363,udp,-,0.000989,35,...,-,-,0,Dd,1,63,1,46,(empty),2011-08-10 09:46:52.841757059
1380089,1.312970e+09,Cgx7314Ab8BC8Biaoc,114.39.152.129,60531,147.32.84.229,13363,udp,-,41.485554,64,...,-,-,0,Dd,2,120,2,92,(empty),2011-08-10 09:46:52.858937025
1373246,1.312970e+09,CllYA58FYp1VP6WK1,46.253.12.1,8192,147.32.84.229,13363,udp,-,0.001306,31,...,-,-,0,Dd,1,59,1,46,(empty),2011-08-10 09:46:52.869766951
1373247,1.312970e+09,CNAhhm20qP13NumDuh,217.44.227.214,12074,147.32.84.229,13363,udp,-,0.001051,34,...,-,-,0,Dd,1,62,1,46,(empty),2011-08-10 09:46:52.870035887
1373248,1.312970e+09,C0UriK1oAe7lujpc2b,147.32.84.59,6881,80.99.214.223,6881,udp,-,-,-,...,-,-,0,D,1,93,0,0,(empty),2011-08-10 09:46:52.885746002
1373265,1.312970e+09,CJe0UmKw4uQwmwXa8,85.171.3.125,9281,147.32.84.229,13363,udp,-,0.092110,140,...,-,-,0,Dd,1,168,1,57,(empty),2011-08-10 09:46:52.885854006
1363835,1.312970e+09,CYY1cJ3dUZ9OZ4P66d,147.32.84.59,2793,90.183.40.197,6881,tcp,-,2.895659,0,...,-,-,0,S,2,96,0,0,(empty),2011-08-10 09:46:52.886269808
1373249,1.312970e+09,COLRQdfKbhhU7tI66,147.32.84.59,6881,122.55.110.78,6881,udp,-,-,-,...,-,-,0,D,1,93,0,0,(empty),2011-08-10 09:46:52.886426926
1373250,1.312970e+09,C7xLaS39FWfkWL4c41,147.32.84.59,6881,178.119.230.190,6881,udp,-,-,-,...,-,-,0,D,1,93,0,0,(empty),2011-08-10 09:46:52.886741877
1373251,1.312970e+09,COfKuN2p2pQRrM0Ibi,147.32.84.59,6881,78.177.38.95,6881,udp,-,-,-,...,-,-,0,D,1,93,0,0,(empty),2011-08-10 09:46:52.886904001


In [54]:
df[1373050:1373100][df[1373050:1373100]["id.orig_h"]=='74.89.223.204']

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,date


In [56]:
df[df["id.orig_h"]=='212.50.71.179']

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,date
2754,1.312962e+09,ClRCaJ2fZFiLn28Rdj,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.000823,32,...,-,-,0,Dd,1,60,1,46,(empty),2011-08-10 07:46:53.047276974
734714,1.312966e+09,CzgkHr1JN8vaQWyLl,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.000418,57,...,-,-,0,Dd,1,85,1,39,(empty),2011-08-10 08:46:03.229327917
1063623,1.312968e+09,CdqCr61KxkRmVt5pfa,212.50.71.179,39678,147.32.84.229,13363,udp,-,27.631593,71,...,-,-,0,Dd,2,127,2,551,(empty),2011-08-10 09:16:37.282680035
2180831,1.312974e+09,Cgr28N4wGFZLBaKg8,212.50.71.179,39678,147.32.84.229,13363,udp,-,82.379258,165,...,-,-,0,Dd,5,305,5,230,(empty),2011-08-10 11:02:42.240203857
2379725,1.312975e+09,CQRvtQ1yJFzYlqig64,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.001243,35,...,-,-,0,Dd,1,63,1,46,(empty),2011-08-10 11:21:45.633074999
2454490,1.312976e+09,CXWdCa2Wlbkb4y7Th5,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.002515,33,...,-,-,0,Dd,1,61,1,46,(empty),2011-08-10 11:28:39.253422022
2487546,1.312976e+09,CpUhp54QhQ9qRG9ZIc,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.000930,35,...,-,-,0,Dd,1,63,1,46,(empty),2011-08-10 11:31:52.594284058
2674341,1.312977e+09,CzjuT83fyqUzvGSBll,212.50.71.179,39678,147.32.84.229,13363,udp,-,23.130713,134,...,-,-,0,Dd,4,246,4,184,(empty),2011-08-10 11:48:21.511261940
2802312,1.312977e+09,C17CTV3zpjWw8mpeyd,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.000867,44,...,-,-,0,Dd,1,72,1,46,(empty),2011-08-10 11:57:34.075811863
3174103,1.312980e+09,CC92z2THluaoB7li6,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.001105,35,...,-,-,0,Dd,1,63,1,46,(empty),2011-08-10 12:32:34.468233109


In [59]:
df_42[df_42["SrcAddr"]=='212.50.71.179']

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/10 09:46:53.047277,3550.182373,udp,212.50.71.179,39678,<->,147.32.84.229,13363,CON,0.0,0.0,12,875,413,flow=Background-UDP-Established
767166,2011/08/10 11:16:37.282680,27.631594,udp,212.50.71.179,39678,<->,147.32.84.229,13363,CON,0.0,0.0,4,734,155,flow=Background-UDP-Established
1501818,2011/08/10 13:02:42.240204,3291.836426,udp,212.50.71.179,39678,<->,147.32.84.229,13363,CON,0.0,0.0,30,2049,1112,flow=Background-UDP-Established
2213056,2011/08/10 14:32:34.468233,3444.411133,udp,212.50.71.179,39678,<->,147.32.84.229,13363,CON,0.0,0.0,20,1474,817,flow=Background-UDP-Established
2669676,2011/08/10 15:34:03.172276,1010.569824,udp,212.50.71.179,39678,<->,147.32.84.229,13363,CON,0.0,0.0,12,2258,962,flow=Background-UDP-Established


In [42]:
df_42.head()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/10 09:46:53.047277,3550.182373,udp,212.50.71.179,39678,<->,147.32.84.229,13363,CON,0.0,0.0,12,875,413,flow=Background-UDP-Established
1,2011/08/10 09:46:53.048843,0.000883,udp,84.13.246.132,28431,<->,147.32.84.229,13363,CON,0.0,0.0,2,135,75,flow=Background-UDP-Established
2,2011/08/10 09:46:53.049895,0.000326,tcp,217.163.21.35,80,<?>,147.32.86.194,2063,FA_A,0.0,0.0,2,120,60,flow=Background
3,2011/08/10 09:46:53.053771,0.056966,tcp,83.3.77.74,32882,<?>,147.32.85.5,21857,FA_FA,0.0,0.0,3,180,120,flow=Background
4,2011/08/10 09:46:53.053937,3427.768066,udp,74.89.223.204,21278,<->,147.32.84.229,13363,CON,0.0,0.0,42,2856,1596,flow=Background-UDP-Established


In [15]:
class_lst = ["background", "normal", "botnet"]
df_42["class"] = df_42.Label.apply(lambda x : list(set(map(str.lower, x.split("=")[1].split("-"))).intersection(set(class_lst)))[0] \
                                           if len(list(set(map(str.lower, x.split("=")[1].split("-"))).intersection(set(class_lst)))) > 0 else '')

In [21]:
df_netflow_botnet = df_42[df_42["class"]=='botnet']

In [55]:
df_netflow_normal = df_42[df_42["class"]=='normal']

In [17]:
df.loc[0]["date"]

Timestamp('2011-08-10 07:46:55.742159843')

In [14]:
with open('./CTU/CTU_42/' + '/weird.log') as f:
    f = f.readlines()
for line in f:
    if line.split("\t")[0]=='#fields':
        w_columns = line.split("\t")[1:]
        for i, c in enumerate(w_columns):
            if len(c.split("\n"))>1:
                w_columns[i] = c.split("\n")[0]
        break

In [15]:
weird_df1 = pd.read_csv('./CTU/CTU_42/' +'weird.log', delimiter="\t", header=None, names=w_columns, skiprows=lambda x:x in range(8), skipfooter=1)
weird_df2 = pd.read_csv('./CTU/CTU_43/' +'weird.log', delimiter="\t", header=None, names=w_columns, skiprows=lambda x:x in range(8), skipfooter=1)
weird_df3 = pd.read_csv('./CTU/CTU_50/' +'weird.log', delimiter="\t", header=None, names=w_columns, skiprows=lambda x:x in range(8), skipfooter=1)

/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  
/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
total_werid_df = pd.concat([weird_df1, weird_df2, weird_df3])

In [18]:
weird_df["date"] = pd.to_datetime(weird_df["ts"], unit='s')
# weird_df["date"] = weird_df["date"].dt.strftime('%Y-%m-%d:%h-%m-%s')
# weird_df["date"] = weird_df["date"].dt.strftime('%r')
# .strftime('% d % m % Y, % r') 

In [19]:
weird_df["date"][0]

Timestamp('2011-08-10 07:46:53.137347937')

In [36]:
len(weird_df)

18732

In [53]:
len(weird_df["id.orig_h"].unique())

2934

In [17]:
Counter(df[df["id.orig_h"]=='147.32.84.165'].service)

Counter({'http': 1195,
         'dns': 8850,
         '-': 21072,
         'http,irc': 30,
         'smtp': 64,
         'irc': 1,
         'ssl': 63})

In [18]:
len(df[df["id.orig_h"]=='147.32.84.165'])

31275

In [76]:
len(weird_df[weird_df["id.orig_h"]=='147.32.84.165'])

177

In [71]:
botnet_name = weird_df[weird_df["id.orig_h"]=='147.32.84.165'].name.unique()

In [81]:
botnet_cnt = Counter(weird_df[weird_df["id.orig_h"]=='147.32.84.165'].name)

In [87]:
botnet_cnt

Counter({'irc_line_too_short': 31,
         'irc_invalid_line': 30,
         'bad_HTTP_request': 30,
         'empty_http_request': 30,
         'NUL_in_line': 7,
         'line_terminated_with_single_CR': 6,
         'bad_TCP_checksum': 1,
         'dns_unmatched_reply': 14,
         'premature_connection_reuse': 28})

In [82]:
weird_cnt = Counter(weird_df.name)

In [86]:
for i in botnet_cnt.keys():
    print(i, weird_cnt[i])

irc_line_too_short 31
irc_invalid_line 30
bad_HTTP_request 2437
empty_http_request 73
NUL_in_line 7
line_terminated_with_single_CR 1527
bad_TCP_checksum 23
dns_unmatched_reply 209
premature_connection_reuse 162


In [75]:
len(weird_df[weird_df["name"].isin(botnet_name)]["id.orig_h"].unique())

1989

In [63]:
df[df["id.orig_h"].isin(netflow_normal_sradd)]

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,date
1038,1.312962e+09,C7yyBw1pzvuQPOpLnl,147.32.84.59,51790,147.32.3.93,443,tcp,-,20.360263,11284,...,-,-,0,dDaAfFR,54,13444,79,66383,(empty),2011-08-10 07:46:53.058759928
6,1.312962e+09,CvT8o91GKgX2u02Qjj,147.32.84.59,49382,74.200.246.228,80,tcp,-,0.000307,0,...,-,-,0,fAR,2,80,1,40,(empty),2011-08-10 07:46:53.082381010
1864,1.312962e+09,CBKF973TvH9vgLikOh,147.32.84.59,49500,94.124.104.196,80,tcp,-,37.925822,0,...,-,-,0,dAFR,187,7480,1734,2601000,(empty),2011-08-10 07:46:53.093291998
3906996,1.312962e+09,ChZ1NQCz9tLKtJMH8,147.32.84.59,49238,74.125.232.215,443,tcp,-,21215.302978,1842979,...,-,-,0,ADadfR,3805,1995179,5279,2399250,(empty),2011-08-10 07:46:53.125001907
742,1.312962e+09,CGJerK1TerDXGjkBL1,147.32.86.111,58314,147.32.1.20,53,udp,dns,1.553285,31,...,-,-,0,Dd,1,59,1,249,(empty),2011-08-10 07:46:53.146586895
108,1.312962e+09,CA4O0l31KWI0fAWN49,147.32.86.111,54230,147.32.80.9,53,udp,dns,-,-,...,-,-,0,d,0,0,1,127,(empty),2011-08-10 07:46:53.160043001
109,1.312962e+09,CBa0nD4LYNZ7XQS6Df,147.32.86.111,55314,147.32.80.9,53,udp,dns,0.466502,38,...,-,-,0,Dd,1,66,1,168,(empty),2011-08-10 07:46:53.161578894
82011,1.312962e+09,C2KNPr1jlprVhO7Erc,147.32.84.59,57601,69.63.190.10,443,tcp,-,557.652815,40637,...,-,-,0,ADdaFf,186,48083,327,322863,(empty),2011-08-10 07:46:53.178426981
2794,1.312962e+09,Cgofqx1zbk0PDSSmB,147.32.84.59,6881,188.11.68.124,6881,udp,-,-,-,...,-,-,0,D,1,93,0,0,(empty),2011-08-10 07:46:53.246263027
13,1.312962e+09,CsKvft1BE3QdBnnDh,147.32.84.59,2289,151.64.186.190,6881,tcp,-,-,-,...,-,-,0,S,1,48,0,0,(empty),2011-08-10 07:46:53.374593973


In [65]:
df[df["id.orig_h"].isin(netflow_normal_sradd_lst)]

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,date
198,1.312962e+09,CiMywE4sDhydPOOHxf,147.32.84.134,40693,147.32.80.9,53,udp,dns,0.000291,30,...,-,-,0,Dd,1,58,1,121,(empty),2011-08-10 07:46:54.779124975
199,1.312962e+09,CRxGdCYj36gMMSoPl,147.32.84.134,47415,147.32.80.9,53,udp,dns,0.000321,30,...,-,-,0,Dd,1,58,1,161,(empty),2011-08-10 07:46:54.779640913
41,1.312962e+09,CemzmX1G7scb2e0LU2,147.32.84.134,45718,88.86.100.176,80,tcp,http,0.006885,119,...,-,-,0,ShADadfF,5,387,5,602,(empty),2011-08-10 07:46:54.780267954
4060734,1.312962e+09,Cm3oNZ3keSu5yd13k,147.32.84.134,44476,74.125.232.216,443,tcp,-,22021.573748,923748,...,-,-,0,dADa,4077,1135752,3815,742023,(empty),2011-08-10 07:46:56.341127872
4054408,1.312962e+09,CQdw2p3wUEAkatz5M8,147.32.84.164,39177,74.125.232.216,443,tcp,-,21993.241867,1420326,...,-,-,0,DadAfF,9051,1890398,9536,2996738,(empty),2011-08-10 07:46:56.955681086
344,1.312962e+09,CNs4c8wAjxvyLEex6,147.32.84.134,36480,147.32.80.9,53,udp,dns,0.000318,30,...,-,-,0,Dd,1,58,1,121,(empty),2011-08-10 07:46:57.796718836
345,1.312962e+09,CKPwPX3roCVN0lOPYb,147.32.84.134,35039,147.32.80.9,53,udp,dns,0.000230,30,...,-,-,0,Dd,1,58,1,161,(empty),2011-08-10 07:46:57.797273159
93,1.312962e+09,CmRbpzTr18o9Kgke7,147.32.84.134,45719,88.86.100.176,80,tcp,http,0.015850,119,...,-,-,0,ShADadfF,5,387,5,602,(empty),2011-08-10 07:46:57.797696114
641,1.312962e+09,C4cHwQN0caSy19yo5,147.32.84.134,52162,147.32.80.9,53,udp,dns,0.000299,30,...,-,-,0,Dd,1,58,1,121,(empty),2011-08-10 07:47:00.820935011
642,1.312962e+09,CABhz73X2AJkWpnWlj,147.32.84.134,50852,147.32.80.9,53,udp,dns,0.000247,30,...,-,-,0,Dd,1,58,1,161,(empty),2011-08-10 07:47:00.821463108


In [45]:
weird_df[18400:18450]

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,name,addl,notice,peer,date
18400,1.312984e+09,Cr5cc01zyP15aJLJNa,147.32.84.59,60846,80.76.149.146,80,unescaped_special_URI_char,-,F,bro,2011-08-10 13:48:07.360406160
18401,1.312984e+09,CjvUPlJiUVtLr5YR5,147.32.84.59,51720,146.102.17.65,80,HTTP_version_mismatch,-,F,bro,2011-08-10 13:48:08.407361984
18402,1.312984e+09,CEkSI711WykK8OrFrj,151.67.137.154,13470,147.32.85.56,80,bad_HTTP_request,-,F,bro,2011-08-10 13:48:08.837347031
18403,1.312984e+09,-,-,-,-,-,dns_unmatched_msg,-,F,bro,2011-08-10 13:48:09.870162010
18404,1.312984e+09,Cy1WvW1Im74GMpxIQ2,147.32.84.59,54692,76.13.114.90,80,unescaped_special_URI_char,-,F,bro,2011-08-10 13:48:12.472079992
18405,1.312984e+09,CNfo5e1bDgoOHz72H,147.32.84.59,52893,76.13.114.90,80,unescaped_special_URI_char,-,F,bro,2011-08-10 13:48:12.663775921
18406,1.312984e+09,CKR4tBc0eoDFHj8S5,147.32.87.181,54588,184.168.120.203,80,active_connection_reuse,-,F,bro,2011-08-10 13:48:17.531489134
18407,1.312984e+09,CPFX7F3ms08VIxAUfb,90.191.72.53,65216,147.32.84.171,6881,window_recision,-,F,bro,2011-08-10 13:48:18.328930140
18408,1.312984e+09,ChacRT32tPQEA79uzd,109.119.148.181,50820,147.32.84.229,80,bad_HTTP_request,-,F,bro,2011-08-10 13:48:19.728653908
18409,1.312984e+09,ChacRT32tPQEA79uzd,109.119.148.181,50820,147.32.84.229,80,line_terminated_with_single_CR,-,F,bro,2011-08-10 13:48:19.728653908


In [33]:
df_netflow_botnet = df_netflow_botnet.sort_values(by=['StartTime'])

In [47]:
df_netflow_botnet["SrcAddr"].unique()

array(['147.32.84.165'], dtype=object)

In [60]:
len(df_netflow_botnet)

40961

In [59]:
len(df_netflow_normal)

30387

In [66]:
df_42[df_42["SrcAddr"].isin(netflow_normal_sradd_lst)]

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label,class
49,2011/08/10 09:46:53.160043,0.000000,udp,147.32.80.9,53,->,147.32.86.111,54230,INT,0.0,NaN,1,141,141,flow=From-Normal-V42-UDP-CVUT-DNS-Server,normal
307,2011/08/10 09:46:54.045343,0.000000,udp,147.32.80.9,53,->,147.32.86.111,58314,INT,0.0,NaN,1,247,247,flow=From-Normal-V42-UDP-CVUT-DNS-Server,normal
394,2011/08/10 09:46:54.460552,0.000000,udp,147.32.80.9,53,->,147.32.86.111,54823,INT,0.0,NaN,1,173,173,flow=From-Normal-V42-UDP-CVUT-DNS-Server,normal
495,2011/08/10 09:46:54.779125,0.000291,udp,147.32.84.134,40693,<->,147.32.80.9,53,CON,0.0,0.0,2,207,72,flow=From-Normal-V42-Jist,normal
496,2011/08/10 09:46:54.779641,0.000321,udp,147.32.84.134,47415,<->,147.32.80.9,53,CON,0.0,0.0,2,247,72,flow=From-Normal-V42-Jist,normal
497,2011/08/10 09:46:54.780268,0.007678,tcp,147.32.84.134,45718,->,88.86.100.176,80,FSPA_FSPA,0.0,0.0,10,1129,457,flow=From-Normal-V42-Jist,normal
979,2011/08/10 09:46:56.955681,3585.430176,tcp,147.32.84.164,39177,<?>,74.125.232.216,443,PA_PA,0.0,0.0,2514,509187,219986,flow=From-Normal-V42-Grill,normal
1168,2011/08/10 09:46:57.796719,0.000318,udp,147.32.84.134,36480,<->,147.32.80.9,53,CON,0.0,0.0,2,207,72,flow=From-Normal-V42-Jist,normal
1169,2011/08/10 09:46:57.797273,0.000230,udp,147.32.84.134,35039,<->,147.32.80.9,53,CON,0.0,0.0,2,247,72,flow=From-Normal-V42-Jist,normal
1170,2011/08/10 09:46:57.797696,0.016656,tcp,147.32.84.134,45719,->,88.86.100.176,80,FSPA_FSPA,0.0,0.0,10,1129,457,flow=From-Normal-V42-Jist,normal


In [62]:
netflow_normal_sradd = df_netflow_normal["SrcAddr"].unique()

In [64]:
netflow_normal_sradd_lst = ['147.32.84.170', '147.32.84.164', '147.32.84.134', '147.32.87.36', '147.32.80.9', '147.32.87.11']

In [57]:
df_netflow_normal[df_netflow_normal["SrcAddr"]=='147.32.86.111']

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label,class
107473,2011/08/10 09:59:08.579300,0.033198,tcp,147.32.86.111,1849,->,74.125.218.80,80,FSRPA_FSPA,0.0,0.0,11,1264,647,flow=Normal-V42-HTTP-windowsupdate,normal


In [35]:
len(df_netflow_botnet)

40961

In [32]:
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,date
2754,1.312962e+09,ClRCaJ2fZFiLn28Rdj,212.50.71.179,39678,147.32.84.229,13363,udp,-,0.000823,32,...,-,-,0,Dd,1,60,1,46,(empty),2011-08-10 07:46:53.047276974
2755,1.312962e+09,CQn1un1RuyREXcKaLc,84.13.246.132,28431,147.32.84.229,13363,udp,-,0.000883,33,...,-,-,0,Dd,1,61,1,46,(empty),2011-08-10 07:46:53.048842907
4,1.312962e+09,C5h5vWjIYesiHr6,147.32.86.194,2063,217.163.21.35,80,tcp,-,0.000326,0,...,-,-,0,fA,1,40,1,40,(empty),2011-08-10 07:46:53.049894810
5,1.312962e+09,CYJRZ54aNSEli4Y55c,83.3.77.74,32882,147.32.85.5,21857,tcp,-,0.000154,0,...,-,-,0,FfA,2,80,1,40,(empty),2011-08-10 07:46:53.053771019
2756,1.312962e+09,CeylmrWwzzpAOG687,74.89.223.204,21278,147.32.84.229,13363,udp,-,0.001000,34,...,-,-,0,Dd,1,62,1,46,(empty),2011-08-10 07:46:53.053936958


In [13]:
df_42 = pd.read_csv('CTU_42/capture20110810.binetflow')

In [12]:
df_42.head()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/10 09:46:53.047277,3550.182373,udp,212.50.71.179,39678,<->,147.32.84.229,13363,CON,0.0,0.0,12,875,413,flow=Background-UDP-Established
1,2011/08/10 09:46:53.048843,0.000883,udp,84.13.246.132,28431,<->,147.32.84.229,13363,CON,0.0,0.0,2,135,75,flow=Background-UDP-Established
2,2011/08/10 09:46:53.049895,0.000326,tcp,217.163.21.35,80,<?>,147.32.86.194,2063,FA_A,0.0,0.0,2,120,60,flow=Background
3,2011/08/10 09:46:53.053771,0.056966,tcp,83.3.77.74,32882,<?>,147.32.85.5,21857,FA_FA,0.0,0.0,3,180,120,flow=Background
4,2011/08/10 09:46:53.053937,3427.768066,udp,74.89.223.204,21278,<->,147.32.84.229,13363,CON,0.0,0.0,42,2856,1596,flow=Background-UDP-Established


In [13]:
df_42["StartTime"][0]

'2011/08/10 09:46:53.047277'

In [27]:
len(df_42)

1121076